This notebook uses langchain to create a knowledge graph based on the wikipedia page about the History of Portugal.

### Import

In [124]:
import langchain
from langchain.llms import OpenAI
from langchain import ConversationChain
from langchain.memory import ConversationKGMemory
import os


os.environ["OPENAI_API_KEY"] = "" ##INTRODUCE YOUR KEY HERE

### Create graph

In [126]:
#Create knowledge graph
#https://medium.com/@bratanic-tomaz/constructing-knowledge-graphs-from-text-using-openai-functions-096a6d010c17

#The following code will instantiate a LangChain wrapper to connect to Neo4j Database.
#https://python.langchain.com/docs/use_cases/graph/graph_cypher_qa

from langchain.graphs import Neo4jGraph
#Complete here with the information of your DB
url = "neo4j+s://4bbe0240.databases.neo4j.io"
username ="neo4j"
password = "DC7rTr-Pnqa_u7idmZavGqsKUZGSb3akeEJ2BsJCk5w"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)


[#0000]  _: <POOL> created, routing address IPv4Address(('4bbe0240.databases.neo4j.io', 7687))
[#0000]  _: <WORKSPACE> resolve home database
[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('4bbe0240.databases.neo4j.io', 7687))
[#0000]  _: <RESOLVE> in: 4bbe0240.databases.neo4j.io:7687
[#0000]  _: <RESOLVE> dns resolver out: 35.189.250.174:7687
[#0000]  _: <POOL> _acquire router connection, database=None, address=ResolvedIPv4Address(('35.189.250.174', 7687))
[#0000]  _: <POOL> trying to hand out new connection
[#0000]  C: <OPEN> 35.189.250.174:7687


[#C80B]  C: <SECURE> 4bbe0240.databases.neo4j.io
[#C80B]  C: <MAGIC> 0x6060B017
[#C80B]  C: <HANDSHAKE> 0x00040405 0x00020404 0x00000104 0x00000003
[#C80B]  S: <HANDSHAKE> 0x00000405
[#C80B]  C: HELLO {'user_agent': 'neo4j-python/5.14.0 Python/3.11.1-final-0 (win32)', 'routing': {'address': '4bbe0240.databases.neo4j.io:7687'}, 'bolt_agent': {'product': 'neo4j-python/5.14.0', 'platform': 'Windows 10; AMD64', 'language': 'Python/3.11.1-final-0', 'language_details': 'CPython; 3.11.1-final-0 (tags/v3.11.1:a7a450f, Dec  6 2022 19:58:39) [MSC v.1934 64 bit (AMD64)]'}}
[#C80B]  _: <CONNECTION> client state: CONNECTED > AUTHENTICATION
[#C80B]  C: LOGON {'scheme': 'basic', 'principal': 'neo4j', 'credentials': '*******'}
[#C80B]  _: <CONNECTION> client state: AUTHENTICATION > READY
[#C80B]  S: SUCCESS {'server': 'Neo4j/5.13-aura', 'connection_id': 'bolt-34', 'hints': {'connection.recv_timeout_seconds': 60, 'telemetry.enabled': False}}
[#C80B]  _: <CONNECTION> server state: CONNECTED > AUTHENTICA

In [127]:
#Show the schema of the graph
print(graph.schema)


        Node properties are the following:
        []
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [128]:
#Define the structure of information we want to extract from text. LangChain already has definitions of nodes and relationship as Pydantic classes that we can reuse.

from langchain.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship
)
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )


class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

In [129]:
#from OpenAi import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.openai_functions.base import create_structured_output_chain

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
    [(
      "system",
      f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"), 
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.  
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial. 
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination."""),
        ("human", "Use the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

In [130]:
#Some useful functions
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )
     

In [131]:
from langchain.graphs.graph_document import GraphDocument
from langchain.schema.document import Document

def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.run(document.page_content)
    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )
    # Store information into a graph
    graph.add_graph_documents([graph_document])

In [132]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter

# Read the wikipedia article
raw_documents = WikipediaLoader(query="History of Portugal").load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)

# Only take the first the raw_documents
documents = text_splitter.split_documents(raw_documents)

In [133]:
print(documents)

[Document(page_content='The history of Portugal can be traced from circa 400,000 years ago, when the region of present-day Portugal was inhabited by Homo heidelbergensis.\nThe Roman conquest of the Iberian Peninsula, which lasted almost two centuries, led to the establishment of the provinces of Lusitania in the south and Gallaecia in the north of what is now Portugal. Following the fall of Rome, Germanic tribes controlled the territory between the 5th and 8th centuries, including the Kingdom of the Suebi centred in Braga and the Visigothic Kingdom in the south.\nThe 711–716 invasion by the Islamic Umayyad Caliphate conquered the Visigoth Kingdom and founded the Islamic State of Al-Andalus, gradually advancing through Iberia. In 1095, Portugal broke away from the Kingdom of Galicia.  Afonso Henriques, son of the count Henry of Burgundy, proclaimed himself king of Portugal in 1139. The Algarve (the southernmost province of Portugal) was conquered from the Moors in 1249, and in 1255 Lisb

In [134]:
from tqdm import tqdm

for i, d in tqdm(enumerate(documents), total=len(documents)):
    extract_and_store_graph(d)

  0%|          | 0/10 [00:00<?, ?it/s]

[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
[#0000]  _: <POOL> routing aged?, database=None
[#0000]  _: <ROUTING> purge check: last_updated_time=1681209.434709, ttl=0, perf_time=1681232.8377517 => False
[#0000]  _: <POOL> routing aged?, database=neo4j
[#0000]  _: <ROUTING> purge check: last_updated_time=1681209.8993416, ttl=10, perf_time=1681232.839774 => False
[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=True, has_server_for_mode=True, table routers={IPv4Address(('4bbe0240.databases.neo4j.io', 7687))} => False
[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('4bbe0240.databases.neo4j.io', 7687))
[#0000]  _: <RESOLVE> in: 4bbe0240.databases.neo4j.io:7687
[#0000]  _: <RESOLVE> dns resolver out: 35.189.250.174:7687
[#0000]  _: <POOL> _acquire router connection, database='neo4j', address=ResolvedIPv4Address(('35.189.250.174', 7687))
[#C80B]  _: <POOL> picked existing connection bolt-34
[#C80B

 10%|█         | 1/10 [00:10<01:32, 10.29s/it]

[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
[#0000]  _: <POOL> routing aged?, database=None
[#0000]  _: <ROUTING> purge check: last_updated_time=1681209.434709, ttl=0, perf_time=1681279.655005 => True
[#0000]  _: <POOL> dropping routing table for database=None
[#0000]  _: <POOL> routing aged?, database=neo4j
[#0000]  _: <ROUTING> purge check: last_updated_time=1681232.9094354, ttl=10, perf_time=1681279.6580438 => True
[#0000]  _: <POOL> dropping routing table for database=neo4j
[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=True, has_server_for_mode=False, table routers={IPv4Address(('4bbe0240.databases.neo4j.io', 7687))} => False
[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('4bbe0240.databases.neo4j.io', 7687))
[#0000]  _: <RESOLVE> in: 4bbe0240.databases.neo4j.io:7687
[#0000]  _: <RESOLVE> dns resolver out: 35.189.250.174:7687
[#0000]  _: <POOL> _acquire router connection, database='ne

 20%|██        | 2/10 [00:58<04:21, 32.66s/it]

...

 80%|████████  | 8/10 [10:17<03:43, 111.71s/it]

[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
[#0000]  _: <POOL> routing aged?, database=neo4j
[#0000]  _: <ROUTING> purge check: last_updated_time=1681836.3215468, ttl=10, perf_time=1681883.7114127 => True
[#0000]  _: <POOL> dropping routing table for database=neo4j
[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=True, has_server_for_mode=False, table routers={IPv4Address(('4bbe0240.databases.neo4j.io', 7687))} => False
[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('4bbe0240.databases.neo4j.io', 7687))
[#0000]  _: <RESOLVE> in: 4bbe0240.databases.neo4j.io:7687
[#0000]  _: <RESOLVE> dns resolver out: 35.189.250.174:7687
[#0000]  _: <POOL> _acquire router connection, database='neo4j', address=ResolvedIPv4Address(('35.189.250.174', 7687))
[#C80B]  _: <POOL> picked existing connection bolt-34
[#C80B]  _: <POOL> checked re_auth auth=None updated=False force=False
[#C80B]  _: <POOL> handing out e

 90%|█████████ | 9/10 [11:01<01:30, 90.77s/it] 

[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
[#0000]  _: <POOL> routing aged?, database=neo4j
[#0000]  _: <ROUTING> purge check: last_updated_time=1681883.8470209, ttl=10, perf_time=1681941.5434213 => True
[#0000]  _: <POOL> dropping routing table for database=neo4j
[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=True, has_server_for_mode=False, table routers={IPv4Address(('4bbe0240.databases.neo4j.io', 7687))} => False
[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('4bbe0240.databases.neo4j.io', 7687))
[#0000]  _: <RESOLVE> in: 4bbe0240.databases.neo4j.io:7687
[#0000]  _: <RESOLVE> dns resolver out: 35.189.250.174:7687
[#0000]  _: <POOL> _acquire router connection, database='neo4j', address=ResolvedIPv4Address(('35.189.250.174', 7687))
[#C80B]  _: <POOL> picked existing connection bolt-34
[#C80B]  _: <POOL> checked re_auth auth=None updated=False force=False
[#C80B]  _: <POOL> handing out e

100%|██████████| 10/10 [11:59<00:00, 71.98s/it]


### Chatbot

In [142]:
# Query the knowledge graph in a RAG application
#https://python.langchain.com/docs/use_cases/graph/graph_cypher_qa
#https://medium.com/neo4j/context-aware-knowledge-graph-chatbot-with-gpt-4-and-neo4j-d3a99e8ae21e

from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0.9, model="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo"),
    validate_cypher=True, # Validate relationship directions
    verbose=True
)

cypher_chain.run("Who was the first king of Portugal?")


[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
[#0000]  _: <POOL> routing aged?, database=neo4j
[#0000]  _: <ROUTING> purge check: last_updated_time=1682260.2679498, ttl=10, perf_time=1682352.8911973 => True
[#0000]  _: <POOL> dropping routing table for database=neo4j
[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=True, has_server_for_mode=False, table routers={IPv4Address(('4bbe0240.databases.neo4j.io', 7687))} => False
[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('4bbe0240.databases.neo4j.io', 7687))
[#0000]  _: <RESOLVE> in: 4bbe0240.databases.neo4j.io:7687
[#0000]  _: <RESOLVE> dns resolver out: 35.189.250.174:7687
[#0000]  _: <POOL> _acquire router connection, database='neo4j', address=ResolvedIPv4Address(('35.189.250.174', 7687))
[#C80B]  _: <POOL> picked existing connection bolt-34
[#C80B]  _: <POOL> checked re_auth auth=None updated=False force=False
[#C80B]  _: <POOL> handing out e

'The first king of Portugal was Afonso I, also known as Afonso Henriques. He reigned from 1139 until his death in 1185 and played a crucial role in establishing the Kingdom of Portugal.'